In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import selenium
from selenium import webdriver

In [2]:
result = requests.get('http://www.ifaq.gov.sg/pdpc/apps/fcd_faqmain.aspx')
soup = BeautifulSoup(result.content)

In [120]:
main_tree = soup.find_all('a', attrs={'class':'tree_childlink'})
qa_links = [item.get('href') for item in main_tree]

In [135]:
def get_meta(qna_section):
    meta_list = qna_section.find_all(['h2', 'h3'])
    meta_list = [item.text for item in meta_list]
    meta = ', '.join(meta_list)
    return meta+'\n'

def get_questions(qna_section):
    question_list = qna_section.find_all('a', attrs={'title':'View answer'})
    question_list = [item.text.replace('\r', '').replace('\n', '') for item in question_list]
    return question_list

def get_answers(qna_section):
    answer_list = qna_section.find_all('div', attrs={'class':'faq_answer'})
    answer_list = [re.sub(r'\n+', '\n', item.text.replace('\r', '').replace('\xa0', '')).strip() for item in answer_list]
    return answer_list

def get_qa_pair(link):
    soup = BeautifulSoup(requests.get(link).content)
    qna_section = soup.find('div', attrs={'id':'print'})
    meta_string = get_meta(qna_section)
    question_list = get_questions(qna_section)
    answer_list = get_answers(qna_section)
    assert len(question_list)==len(answer_list), 'questions and answers not equal {}, {}'.format(len(question_list), len(answer_list))
    df_scraped = pd.DataFrame({'question':question_list, 'meta':[meta_string]*len(question_list), 'answer':answer_list})
    return df_scraped


In [142]:
df_pdpa = pd.DataFrame(columns=['question', 'meta', 'answer'])
for qa_link in qa_links:
    print(qa_link)
    df_scraped = get_qa_pair(qa_link)
    df_pdpa = df_pdpa.append(df_scraped, ignore_index=True)

http://www.ifaq.gov.sg:80/pdpc/apps/fcd_faqmain.aspx?qst=hRhkP9BzcBImsx2TBbssMsxu7lqt6UJK70a1wAEVmyfiNHrkSWapUoxJrod9%2bbyLC2869whY9PmzDAqIf0khkfdKfkXyUc2p%2bZeN8PHR1T32xrVks%2bZ10FLHQAia7aJUK9mcKZAJ3Hp9lWeoJh5J0jJig821hiii2%2f4Ht%2fkXC1KD6JR4U2%2fjP6hQepW%2f2liM%2fUUMs6i%2fiSeqjrqajoKZ3jq5eY14VOuQ2vBlq6ADR3g%3d
http://www.ifaq.gov.sg:80/pdpc/apps/fcd_faqmain.aspx?qst=hRhkP9BzcBImsx2TBbssMsxu7lqt6UJK70a1wAEVmyfiNHrkSWapUoxJrod9%2bbyLC2869whY9PmzDAqIf0khkfdKfkXyUc2p%2bZeN8PHR1T32xrVks%2bZ10FLHQAia7aJU%2bCgyFXiviMpCWSxFt279TDLEvkR6Q8b5MYDxklzQtgSdzGy58RgwlAsxC7JrrTOiGJPCpr6osu0NmQ1ygidvQHt4GNFv9W6FFLQQGwQbg3g%3d
http://www.ifaq.gov.sg:80/pdpc/apps/fcd_faqmain.aspx?qst=hRhkP9BzcBImsx2TBbssMsxu7lqt6UJK70a1wAEVmyfiNHrkSWapUoxJrod9%2bbyLC2869whY9PmzDAqIf0khkfdKfkXyUc2p%2bZeN8PHR1T32xrVks%2bZ10FLHQAia7aJUHAE%2fS8%2bYogJ8IO0heUp4rD%2bZ6Hb489%2f3ROOITVC9L4KaIDM9LYlQJIhMbCLr9cq31i%2fXh4RlILFKxle1sJWNpg02l8EC6PwYnEd3iU6sUiY%3d
http://www.ifaq.gov.sg:80/pdpc/apps/fcd_faqmain.aspx?qst=hRhkP9BzcBIms

In [143]:
print(len(df_pdpa))
df_pdpa

269


,question,meta,answer
0,What is personal data?,"Organisations, General\n","Personal data refers to data, whether true or ..."
1,When did the PDPA come into force?,"Organisations, General\n",The PDPA was implemented in phases to allow ti...
2,What are the objectives of the PDPA?,"Organisations, General\n",The PDPA aims to safeguard individuals’ person...
3,How does the PDPA benefit business?,"Organisations, General\n",The PDPA will strengthen Singapore's overall e...
4,How will the PDPA impact business costs?,"Organisations, General\n",The provisions of the PDPA were formulated kee...
5,How is the PDPA different from the Spam Contro...,"Organisations, General\n",The Spam Control Act (SCA) sets out a framewor...
6,What is 'publicly available'?,"Organisations, General\n",‘Publicly available’ in relation to personal d...
7,"Can an organisation collect, use or disclose p...","Organisations, General\n",While an organisation may not obtain consent f...
8,Are public databases considered publicly avail...,"Organisations, General\n",Publicly available personal data refers to per...
9,Are randomly generated data considered persona...,"Organisations, General\n","Personal data is defined under the PDPA as ""da..."


In [144]:
df_pdpa.to_csv('../data/pdpa.csv', index=False)

# Scrape legislation
It is weirdly protected, so I had to copy the html out.

In [65]:
# result = requests.get('https://sso.agc.gov.sg/Act/PDPA2012')
f = open('../data/pdpa.html', 'r', encoding='utf-8')
soup = BeautifulSoup(f.read())

In [78]:
all_parts = soup.find_all('td', attrs={'class':'part'})

In [68]:
soup.find_all('td', attrs={'class':'partHdr'})

[<td class="partHdr" id="P1I-he-">PRELIMINARY</td>,
 <td class="partHdr" id="P1II-he-">PERSONAL DATA PROTECTION COMMISSION <br/>AND ADMINISTRATION</td>,
 <td class="partHdr" id="P1III-he-">GENERAL RULES WITH RESPECT TO <br/>PROTECTION OF PERSONAL DATA</td>,
 <td class="partHdr" id="P1IV-he-">COLLECTION, USE AND DISCLOSURE OF <br/>PERSONAL DATA</td>,
 <td class="partHdr" id="P1V-he-">ACCESS TO AND CORRECTION OF <br/>PERSONAL DATA</td>,
 <td class="partHdr" id="P1VI-he-">CARE OF PERSONAL DATA</td>,
 <td class="partHdr" id="P1VII-he-">ENFORCEMENT OF PARTS III TO VI</td>,
 <td class="partHdr" id="P1VIII-he-">APPEALS TO DATA PROTECTION APPEAL COMMITTEE, HIGH COURT AND COURT OF APPEAL</td>,
 <td class="partHdr" id="P1IX-he-">DO NOT CALL REGISTRY</td>,
 <td class="partHdr" id="P1X-he-">GENERAL</td>]

In [73]:
print(len(soup.find_all('td', class_='prov1Hdr')))
print(len(soup.find_all('td', class_='prov1Txt')))

68
68


In [105]:
df_pdpa_legis = pd.DataFrame(columns=['meta', 'answer'])
for part in all_parts[1:]:
    part_header = part.find('td', class_='partHdr').text
    
    provisions = part.find_all('td', class_='prov1Hdr')
    provisions = [item.text for item in provisions]
    prov_texts = part.find_all('td', class_='prov1Txt')
    prov_texts = [re.sub(r'\n+', '\n', item.text.replace('\r', '').replace('\xa0', '')).strip() for item in prov_texts]
    print(len(prov_texts), len(provisions))
    meta_list = [part_header+', '+item for item in provisions]
    print(meta_list)
    df_part = pd.DataFrame({'meta':meta_list, 'answer':prov_texts})
    df_pdpa_legis = df_pdpa_legis.append(df_part, ignore_index=True)


6 6
['PERSONAL DATA PROTECTION COMMISSION AND ADMINISTRATION, Personal Data Protection Commission', 'PERSONAL DATA PROTECTION COMMISSION AND ADMINISTRATION, Functions of Commission', 'PERSONAL DATA PROTECTION COMMISSION AND ADMINISTRATION, Advisory committees', 'PERSONAL DATA PROTECTION COMMISSION AND ADMINISTRATION, Delegation', 'PERSONAL DATA PROTECTION COMMISSION AND ADMINISTRATION, Conduct of proceedings', 'PERSONAL DATA PROTECTION COMMISSION AND ADMINISTRATION, Co-operation agreements']
2 2
['GENERAL RULES WITH RESPECT TO PROTECTION OF PERSONAL DATA, Compliance with Act', 'GENERAL RULES WITH RESPECT TO PROTECTION OF PERSONAL DATA, Policies and practices']
8 8
['COLLECTION, USE AND DISCLOSURE OF PERSONAL DATA, Consent required', 'COLLECTION, USE AND DISCLOSURE OF PERSONAL DATA, Provision of consent', 'COLLECTION, USE AND DISCLOSURE OF PERSONAL DATA, Deemed consent', 'COLLECTION, USE AND DISCLOSURE OF PERSONAL DATA, Withdrawal of consent', 'COLLECTION, USE AND DISCLOSURE OF PERSONAL

In [111]:
len(df_pdpa_legis)
df_pdpa_legis.to_csv('../data/pdpa_legis.csv', index=False)

# Scrape ncov FAQ

In [68]:
url = 'https://www.moh.gov.sg/2019-ncov-wuhan/faqs'

In [69]:
def browser(url):
    driver = webdriver.Chrome("C:/Users/likkh/Downloads/chromedriver_win32/chromedriver.exe")
    driver.get(url)
    return driver

In [70]:
driver = browser(url)
els = driver.find_elements_by_class_name('expand')
for el in els:
    el.click()

ElementClickInterceptedException: Message: element click intercepted: Element <a href="#" class="expand">...</a> is not clickable at point (53, 663). Other element would receive the click: <td style="text-align: left;" width="40" valign="top">...</td>
  (Session info: chrome=80.0.3987.100)


In [71]:
soup = BeautifulSoup(driver.page_source)

In [72]:
questions=[]
answers=[]

divs = soup.find_all(attrs={'class':'show-more'})
for div in divs:
    rows = div.find_all('tr')
    for ii, row in enumerate(rows):
        texts = row.find_all('p')
        try:
            if ii%2:
                answers.append(texts[1].text)
            else:
                questions.append(texts[1].text)
        except:pass

df_ncov = pd.DataFrame({'question':questions, 'meta':['Ministry of Health.']*len(answers),'answer':answers})

In [73]:
df_ncov

,question,meta,answer
0,What are Coronaviruses?,Ministry of Health.,Coronaviruses are a large family of viruses ca...
1,How does COVID-19 spread?,Ministry of Health.,There is human to human transmission of COVID-...
2,What are the symptoms of COVID-19?,Ministry of Health.,The symptoms of COVID-19 infection are similar...
3,What is the treatment for COVID-19?,Ministry of Health.,Supportive treatment is provided based on the ...
4,Is the disease deadly? How is this compared to...,Ministry of Health.,The situation is evolving and many characteris...
5,What is the current public health travel advis...,Ministry of Health.,MOH advises travellers to avoid travel to Hube...
6,"If I defer my travel to mainland China, will t...",Ministry of Health.,You are advised to check with your respective ...
7,"How are the frontline workers, e.g., airline s...",Ministry of Health.,Employees working in the various sectors would...
8,"With the new H5N1 outbreak in Hunan, China, ar...",Ministry of Health.,MOH is currently monitoring the development of...
9,"Can organisations collect, use and disclose pe...",Ministry of Health.,Organisations may collect personal data of vi...


In [78]:
url = 'https://www.moe.gov.sg/faqs-wuhan-coronavirus-infection'

In [79]:
result = requests.get(url)
soup = BeautifulSoup(result.content)

In [80]:
faq_content = soup.find_all(attrs={'class':'sfContentBlock'})[3]

In [81]:
questions=faq_content.find_all('h5')
questions = [item.text for item in questions]

In [83]:
answers=[]
content = ''
for tag in faq_content.find_all(['h5', 'p', 'ul']):
    if tag.name == "h5":
        if content != '\xa0 ':
            answers.append(content)
        content = ''
        pass
    else:
        if 'Back to Top' not in tag.text:
            content+=tag.text+' '
answers.append(content)

In [84]:
df_moe = pd.DataFrame({'question':questions, 'meta':['Ministry of Education.']*len(answers),'answer':answers})

In [85]:
df_moe

,question,meta,answer
0,"With the move to DORSCON Orange, what measures...",Ministry of Education.,We have further enhanced our measures by suspe...
1,Why is my child sent home when he does not see...,Ministry of Education.,Schools conduct temperature-taking exercises d...
2,Will mass events involving staff and stakehold...,Ministry of Education.,"Mass events with staff and stakeholders, but n..."
3,Why implement the social distancing measures now?,Ministry of Education.,We are constantly reviewing our measures to pr...
4,What MOE is doing is not fool proof. What more...,Ministry of Education.,"No measure is 100% fool proof or guaranteed, b..."
5,The World Health Organisation (WHO) has declar...,Ministry of Education.,We review measures and closely monitor the sit...
6,Can schools provide hand sanitizers in all cla...,Ministry of Education.,Personal hygiene is indeed important. But hand...
7,Do the precautionary measures by the Ministry ...,Ministry of Education.,Tuition and enrichment centres are private ent...
8,Why not close schools when DORSCON was changed...,Ministry of Education.,DORSCON Orange means we need to take more prec...
9,Can schools be one of the safest place for stu...,Ministry of Education.,Staff and students are already on extra alert....


In [86]:
df = pd.concat([df_ncov, df_moe], axis=0)
print(len(df_ncov), len(df_moe), len(df))

54 23 77


In [87]:
df

,question,meta,answer
0,What are Coronaviruses?,Ministry of Health.,Coronaviruses are a large family of viruses ca...
1,How does COVID-19 spread?,Ministry of Health.,There is human to human transmission of COVID-...
2,What are the symptoms of COVID-19?,Ministry of Health.,The symptoms of COVID-19 infection are similar...
3,What is the treatment for COVID-19?,Ministry of Health.,Supportive treatment is provided based on the ...
4,Is the disease deadly? How is this compared to...,Ministry of Health.,The situation is evolving and many characteris...
...,...,...,...
18,What if students or staff who are on LOA do no...,Ministry of Education.,There is a difference between an LOA and a Qua...
19,"Are vendors, canteen staff, freelance coaches ...",Ministry of Education.,"Yes. The LOA provisions cover teaching staff, ..."
20,How about students and staff who did not come ...,Ministry of Education.,"As explained earlier, the LOA is a precautiona..."
21,What if students or staff do not declare their...,Ministry of Education.,We urge them to spare a thought for others and...


In [88]:
df.to_csv('../data/covid19.csv', index=False)